In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
df=pd.read_csv('Restaurant.csv')

In [3]:
df.head(5)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [4]:
a,b=df.shape
print('rows :',a)
print('columns :',b)

rows : 9551
columns : 21


In [5]:
df.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [6]:
df.describe()

,Restaurant ID,Country Code,Longitude,Latitude,Average Cost for two,Price range,Aggregate rating,Votes
count,9.551000e+03,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000
mean,9.051128e+06,18.365616,64.126574,25.854381,1199.210763,1.804837,2.666370,156.909748
std,8.791521e+06,56.750546,41.467058,11.007935,16121.183073,0.905609,1.516378,430.169145
min,5.300000e+01,1.000000,-157.948486,-41.330428,0.000000,1.000000,0.000000,0.000000
25%,3.019625e+05,1.000000,77.081343,28.478713,250.000000,1.000000,2.500000,5.000000
50%,6.004089e+06,1.000000,77.191964,28.570469,400.000000,2.000000,3.200000,31.000000
75%,1.835229e+07,1.000000,77.282006,28.642758,700.000000,2.000000,3.700000,131.000000
max,1.850065e+07,216.000000,174.832089,55.976980,800000.000000,4.000000,4.900000,10934.000000


In [7]:
df=df.dropna(axis=1)
df=df.drop(columns=['Locality','Locality Verbose'],axis=1)

In [8]:
df[['Rating text','Rating color']].value_counts()

Rating text  Rating color
Average      Orange          3737
Not rated    White           2148
Good         Yellow          2100
Very Good    Green           1079
Excellent    Dark Green       301
Poor         Red              186
dtype: int64

In [9]:
le= LabelEncoder()
columns = ['Restaurant Name','Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu', 'Rating text','Rating color','Currency','City']
for col in columns:
    df[col]=le.fit_transform(df[col])


In [10]:
df.head(5)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Longitude,Latitude,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,3748,162,73,"Third Floor, Century City Mall, Kalayaan Avenu...",121.027535,14.565443,1100,0,1,0,0,0,3,4.8,0,1,314
1,6304287,3172,162,73,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...",121.014101,14.553708,1200,0,1,0,0,0,3,4.5,0,1,591
2,6300002,2896,162,75,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...",121.056831,14.581404,4000,0,1,0,0,0,4,4.4,1,5,270
3,6318506,4707,162,75,"Third Floor, Mega Fashion Hall, SM Megamall, O...",121.056475,14.585318,1500,0,0,0,0,0,4,4.9,0,1,365
4,6314302,5523,162,75,"Third Floor, Mega Atrium, SM Megamall, Ortigas...",121.057508,14.584450,1500,0,1,0,0,0,4,4.8,0,1,229


In [11]:
df.shape

(9551, 18)

In [12]:
tfidf = TfidfVectorizer(max_features=100)  
for col in ['Address']:
    tfidf_result = tfidf.fit_transform(df[col]).toarray()
    tfidf_df = pd.DataFrame(tfidf_result, columns=[col + "_" + str(i) for i in range(tfidf_result.shape[1])])
    df = pd.concat([df, tfidf_df], axis=1)
    df.drop(col, axis=1, inplace=True)
    

In [13]:
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Longitude,Latitude,Average Cost for two,Currency,Has Table booking,Has Online delivery,...,Address_90,Address_91,Address_92,Address_93,Address_94,Address_95,Address_96,Address_97,Address_98,Address_99
0,6317637,3748,162,73,121.027535,14.565443,1100,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6304287,3172,162,73,121.014101,14.553708,1200,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6300002,2896,162,75,121.056831,14.581404,4000,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6318506,4707,162,75,121.056475,14.585318,1500,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6314302,5523,162,75,121.057508,14.584450,1500,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
x=df.drop(['Aggregate rating','Restaurant ID',], axis=1)
y=df['Aggregate rating']


In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
lreg=LinearRegression()

In [17]:
lreg.fit(x_train,y_train)

LinearRegression()

In [18]:
model=XGBRegressor()

In [19]:
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [20]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)  
rf.fit(x_train, y_train)

RandomForestRegressor(random_state=42)

In [21]:
dt = DecisionTreeRegressor(random_state=42)
dt.fit(x_train, y_train)

DecisionTreeRegressor(random_state=42)

In [22]:
models = {
    "Linear Regression": lreg,
    "XGBRegressor": model,
    "Random Forest": rf,
    "Decision Tree": dt
}


In [23]:
def evaluate_models(models, x_train, y_train, x_test, y_test):
    results = {}  
    for name, model in models.items():
       
       
        y_pred = model.predict(x_test)
        
       
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
       
        results[name] = {"MSE": mse, "R2": r2}
        
     
        print(f"Model: {name}")
        print(f"Mean Squared Error: {mse:.2f}")
        print(f"R-squared: {r2:.2f}\n")
        
    return results

model_results = evaluate_models(models, x_train, y_train, x_test, y_test)


Model: Linear Regression
Mean Squared Error: 1.16
R-squared: 0.49

Model: XGBRegressor
Mean Squared Error: 0.03
R-squared: 0.99

Model: Random Forest
Mean Squared Error: 0.03
R-squared: 0.99

Model: Decision Tree
Mean Squared Error: 0.06
R-squared: 0.98

